## Notebook Content

In this notebook, deep text cleaning for the ready dataset that comes from Kaggle then starts topic modeling and multiple iterations of removing the domain-specific words, then finally labeling the data to start supervised learning procedures.

## Import Libraries

In [1]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem import WordNetLemmatizer
import pandas as pd
from collections import Counter
import re
import pickle
import string
from sklearn.decomposition import NMF
import spacy
import pandas as pd
import numpy as np
import scipy.sparse as ss
from sklearn.feature_extraction.text import CountVectorizer
from corextopic import corextopic as ct
%matplotlib inline

### Methods to Use Amoung File

In [2]:
def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return ' '.join(lemmas)

In [3]:
def only_word(list_):
    """
    This method is to return list of words only in the passed list.
    """
    list_to_return = []
    for ele in list_:
        if len(ele) >= 3:
            list_to_return.append(ele)
    return ' '.join(list_to_return)

In [4]:
def clean_method(text):
    """This method depends on lemmatize_verbs, remove_stopwords and clean_text to apply them on passed parameter"""
    if len(text) <= 100:
        return 'd'
    stopwords = nltk.corpus.stopwords.words('english')
#     text = text.lower() 
    text = text.replace('\\n', ' ').replace('\\r', ' ').replace('\n', ' ').replace('\r', ' ').replace('\xef', ' ').replace('\\xef', ' ')
    text = re.sub('http\S+\s*', ' ', text)
    text = re.sub(r'(.)\1+', r'\1', text)
    text = re.sub('[^a-zA-Z]+', ' ', text)
    text = only_word(text.split())
    text = lemmatize_verbs(text.split())
    text = [word for word in text.split() if word not in stopwords]
    return ' '.join(text)

In [5]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    """...."""
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [6]:
def get_freq_word(TEXT_list):
    """This method get count the frequency of each word in the passed parameter"""
    Vocab_list = ' '.join(TEXT_list)
    temp_dict = Counter(Vocab_list.split()).items()
    freq_df = pd.DataFrame(temp_dict)
    freq_df = freq_df.sort_values(1, ascending = False)
    return freq_df

In [7]:
def write_pickle(file_name, df_to_write):
    """This method is to save the passed dataframe as a pickle file with the passed name"""
    with open(file_name, 'wb') as handle:
        pickle.dump(df_to_write, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [8]:
def read_pickle(file_name):
    """This method is to read the passed file name"""
    with open(file_name , 'rb') as handle:
        return pickle.load(handle)

In [9]:
def write_pickle(file_name, df_to_write):
    """This method is to save the passed dataframe as a pickle file with the passed name"""
    with open(file_name, 'wb') as handle:
        pickle.dump(df_to_write, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [10]:
def remove_domain_stop_words(stop_words, df_edit):
    """Remove domain specific stop words from the passed parameter"""
    df_after = df_edit.copy()
    for i in range(len(df_after)):
        for word in stop_words:
            pattern = r'\b' + word + r'\b'
            new_text = re.sub(pattern,'', df_after.TEXT[i])
            df_after.TEXT[i] = new_text
    return df_after

### Read Data from CSV

In [11]:
data = pd.read_csv('resume_dataset.csv')
data2 = pd.read_csv('resume_dataset2.csv')
data = pd.concat([data, data2])

### Drop Unnecessary Column

In [12]:
# Drop Category and ID columns since we want to 
# apply unsupervised learning procedures
data = data.drop(['Category', 'ID'],axis=1) 
data.columns = ['TEXT']
data.index = range(0, len(data))

In [13]:
data

,TEXT
0,Skills * Programming Languages: Python (pandas...
1,Education Details \r\nMay 2013 to May 2017 B.E...
2,"Areas of Interest Deep Learning, Control Syste..."
3,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...
1383,"b""Free Flight Attendant Resume\nDarlene Flint\..."
1384,b'Corporate Flight Attendant Resume\nCAITLIN F...
1385,"b'MAJOR CONRAD A. PREEDOM\n2354 Fairchild Dr.,..."
1386,b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@em...


### Explore Data

In [14]:
## this step is to display all text in each row
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

/var/folders/5w/wm8hgb2j05790d69l1zpj_br0000gn/T/ipykernel_26605/2084993638.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [15]:
data.sample(2, random_state = 1)

TEXT
255  b'Xochil Herrera Scheer\n\n1307 S. WOLF ROAD\nHILLSIDE, ILLINOIS 60162\nCONTACTME@XOCHIL.COM\n708.790.5190\nlinkedin.com/in/xochil\n\nSkills\n\nFASHION TECHNICAL: Product Development for Apparel (Sportswear, Activewear, Knits) and Accessories, Pattern Making,\nGrading, Sewing, Technical Design, Fabric Sourcing and Development, Textile Knowledge, Leather and Fur Construction,\nAdvanced Tailoring and Garment Redesign\nCOMPUTER: Adobe Photoshop and Illustrator, Optitex Pattern Design, Gerber Accumark Pattern Design, Microsoft Office\nMANAGEMENT/CUSTOMER SERVICE: Managing employees and contractors, project schedules and operations, sales, inventory\nand pricing. Experienced in product development, merchandising, retail buying, and event planning.\n\nFreelance Experience\n\n2004-Present\nXochil Herrera Scheer Fashion Design Services: Pattern making and sample sewing for designers and\nApparel companies. Designed sports wear and evening wear for clients under independent label. Services includes pattern making,\ntechnical design, fabric sourcing, line development, production samples, sewing and tailoring.\nClients/Projects: Diamond MMA, Rayjus Outdoors, Borris Powell, Glass House Shirtmakers, Lauren Lein, Crescendo Apparel,\nBlake Standard, Doc Froc, Cotton Candy Couture, Smashing Golf & Tennis, The Giving Bride, Ellie Day, Skate Armor, Pine & Plastic,\nCasual Friday Apparel\n2009-2011\nThe Image Studios: Tailoring and advanced clothing re-design services for professional client wardrobes.\nWorked directly with stylists and image consultants to achieve strong results for client\'s "personal branding."\n2005\nHouse of Blues - Chicago: Designed and constructed cocktail uniforms (Oct. 2005) and women\xe2\x80\x99s shirt\n(Jan. 2005) for restaurant employees and for sale in the Company Store.\n\nJob Experience\n\nThe Apparel Agency\nChicago, IL\nProduct Development Manager\nWorks together with a team of professionals offering product development services, business and brand strategy, and production\nservices. Works directly with clients, comprehensively throughout each stage of development. Work includes fabric and material\nsourcing and development, technical design, pattern making, and product design. Apparel and accessory sewn products are\nprepared for prototyping and production. Manages support staff, internship program, and works directly with fabric mills, vendors\nand factory sub-contractors. Attends domestic and international trade shows, including TexWorld USA, Premiere Vision, Indigo,\nKingpins, TexWorld Paris, Premiere Vision Plur/El Paris, Modamont, TexProcess Americas.\n2012-2014\n\nClients/Projects: FOMO Activewear, Hank USA, Moxie Cycling, Robyn Glaser Home, 1420 Collection, Stella Blu Clothing,\nAll The Numbers, Molly Ades Maternity, Iguana Med, Michael Alan Stein, Kore Extreme, Hershey\'s Chocolate World\nTurley Road\nChicago, IL\nPattern Maker\nWorked with owner and creative director to design new styles, and revised existing patterns. Managed pattern library. Graded all\npatterns to specified sizes, and cut for small production. Sewed fit samples for new styles and worked with sewing contractor.\n2009-2012\n\nEliana Lily Chicago and Blo Salon and Spa\nChicago, IL\nDesign and Restyling Manager; promoted to General Manager\nManaged Custom Clothing, Restyling and Handbag department, including design, patterns, fabric selection, construction and\nfinishing. Handled vendor relations, inventory, pricing, customer service. Assisted in retail buying. Oversaw up to 15 employees and\ninternship program. Involved in event planning partnering with organizations, charities, and fashion/trunk shows including Lincoln\nPark Chamber of Commerce, Poor Little Rich Girls, Yelp.com, and In Style magazine. Created all promotional materials including\nsignage, posters, flyers, mailings.\n2006-2009\n\nThe Illinois Institute of Art - Chicago\nChicago, IL\nFashion Department Tutor\nHelped students in sewing, pattern making, draping a

In [16]:
data.head(2)

TEXT
0  Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\nData Science Assurance AssociateData Science Assurance Associate - Ernst & Young LLPSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigations and Dispute Services   Assurance\r\nTECHNOLOGY ASSISTED REVIEW\r\nTAR (Technology Assisted Review) assists in accelerating the review process and run analytics and generate reports.\r\n* Core member of a team helped in developing automated review platform tool from scratch for assisting E discovery domain, this tool implements predictive coding and topic modelling by automating reviews, resulting in reduced labor costs and time spent during the lawyers review.\r\n* Understand the end to end flow of the solution, doing research and development for classification models, predictive analysis and mining of the information present in text data. Worked on analyzing the outputs and precision monitoring for the entire tool.\r\n* TAR assists in predictive coding, topic modelling from the evidence by following EY standards. Developed the classifier models in order to identify "red flags" and fraud-related issues.\r\n\r\nTools & Technologies: Python, scikit-learn, tfidf, word2vec, doc2vec, cosine similarity, NaÃ¯ve Bayes, LDA, NMF for topic modelling, Vader and text blob for sentiment analysis. Matplot lib, Tableau dashboard for reporting.\r\n\r\nMULTIPLE DATA SCIENCE AND ANALYTIC PROJECTS (USA CLIENTS)\r\nTEXT ANALYTICS - MOTOR VEHICLE CUSTOMER REVIEW DATA * Received customer feedback survey data for past one year. Performed sentiment (Positive, Negative & Neutral) and time series analysis on customer comments across all 4 categories.\r\n* Created heat map of terms by survey category based on frequency of words * Extracted Positive and Negative words across all the Survey categories and plotted Word cloud.\r\n* Created customized tableau dashboards for effective reporting and visualizations.\r\nCHATBOT * Developed a user friendly chatbot for one of our Products which handle simple questions about hours of operation, reservation options and so on.\r\n* This chat bot serves entire product related questions. Giving overview of tool via QA platform and also give recommendation responses so that user question to build chain of relevant answer.\r\n* This too has intelligence to build the pipeline of questions as per user requirement and asks the relevant /recommended questions.\r\n\r\nTools & Technologies: Python, Natural language processing, NLTK, spacy, topic modelling, Sentiment analysis, Word Embedding, scikit-learn, JavaScript/JQuery, SqlServer\r\n\r\nINFORMATION GOVERNANCE\r\nOrganizations to make informed decisions about all of the information they store. The integrated Information Governance portfolio synthesizes intelligence across unstructured data sources and facilitates action to ensure organizations are best positioned to counter information risk.\r\n* Scan data from multiple sources of formats and parse different file formats, extract Meta data information, push results for indexing elastic search and created customized, interactive dashboards using kibana.\r\n* Preforming ROT Analysis on the data which give information of data which helps identify conten

In [17]:
data.tail(2)

TEXT
1386  b'STACY SAMPLE\n\n702 800-0000 cell\n\n0000@email.com\n\nQualifications\n\nFlight Time\n\nFlight Ratings\nFAA ATP \xe2\x80\x93 Airline Transport Pilot\nCFI / Rotorcraft-Helicopter\nS-70, S-92 Type-ratings\nMedical\nFAA Class 1\n\nExperience\nSikorsky S-76 & S-92 Test Pilot\nJul 2007 \xe2\x80\x93 Present\n\nTotal\n\n6867\n\nPilot in Command\nPilot\nHelicopter Multi-Engine\nCross Country\nInstrument IFR/Hood\nActual Weather\n\n5673\n1194\n4660\n3847\n832\n369\n\nNight\nNight Vision Goggle\nOffshore/Overwater\nExternal Load\nEMS\nMountain\n\n1342\n528\n574\n263\n541\n2693\n\nSikorsky Aircraft Corporation \xe2\x80\x93 Coatesville, PA\nS-76 and S-92 Production Test Pilot \xe2\x80\x93 Maintenance test pilot, engineering\ntest flight, civil and foreign military flight instructor, customer\ndemonstrations, international IFR, VFR, and overwater experience.\n\nBell 430 Corporate IFR Captain\nJan 2006 \xe2\x80\x93 Jun 2007\n\nElite Aviation \xe2\x80\x93 Los Angeles, CA\nSingle Pilot Bell 430 Corporate IFR Captain \xe2\x80\x93 Off-airport landing,\nlogistics, and air/ground security. Company contract included occasional\nScreen Actors Guild motion picture work. Part 91, 135.\n\nSikorsky S-76 EMS Captain\nOct 2003 \xe2\x80\x93 Dec 2005\n\nChildren\xe2\x80\x99s Hospital Los Angeles / Helinet Aviation \xe2\x80\x93Van Nuys, CA\nSingle Pilot S-76 EMS Captain \xe2\x80\x93 Mountainous, rooftop and confined\nareas. Off-day schedule included AS350 & S-76 charter, corporate\ntransport, film work, ENG, and maintenance test flight. Part 91,135.\n\nAS 350 Charter / Tour Pilot\nJan 2002 \xe2\x80\x93 Sep 2003\n\nSundance Helicopters \xe2\x80\x93 Las Vegas, NV and Grand Canyon, AZ\nEurocopter AS 350 Charter Pilot \xe2\x80\x93 Tour, charter, and maintenance flights.\nHigh temp/DA, and high canyon winds at max gross weight, above 5000ft\nmountainous terrain. Part 91,135.\n\nSikorsky S-70 & Bell 205 Pilot\nJan 1994 \xe2\x80\x93 Nov 2001\n\nUnited States Army \xe2\x80\x93 US and the Middle East\nS-70 and Bell 205 Pilot in Command \xe2\x80\x93 Night vision goggle, low level,\nexternal load, rappel, fast rope, parachute drop, drug intervention,\noverwater, medevac, pinnacle, confined area, desert, high temp/DA, and\nabove 10,000ft mountainous terrain.\n\nEducation\nManufacturer Approved Courses\nSikorsky 70 / 76 / 92 / Bell 430\nMaster of Aeronautical Science\n2014\nBachelor of Science Aeronautics\n2010\nUH-1 Huey / UH-60 Black Hawk\n1994-1996\n\nFlight Safety International\nInitial type-ratings and recurrent training\nEmbry-Riddle Aeronautical University\nSpecialization in Aviation/Aerospace Education Technology\nEmbry-Riddle Aeronautical University\nSumma cum laude\nU.S. Army Flight School\nFort Rucker, AL\nBell 205 (UH-1) and Sikorsky S-70 (UH-60) aircraft qualifications'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388 entries, 0 to 1387
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TEXT    1388 non-null   object
dtypes: object(1)
memory usage: 11.0+ KB


In [19]:
#Checking for nulls
data.isnull().sum()

TEXT    0
dtype: int64

In [20]:
#Checking for duplicates
print((data.duplicated('TEXT').sum()))

131


In [21]:
#drop duplicates
data.drop_duplicates('TEXT', inplace = True)

### Start Resume Text Cleaning

In [22]:
# clean text and any empty text will return 'd'
data.index = range(0, len(data))
data['TEXT'] = data['TEXT'].apply(lambda x: clean_method(x))

In [23]:
# Remove empty text
data = data[~(data.TEXT == 'd')]

In [24]:
data.head(2) # data after cleaning

,TEXT
0,Skils Programing Languages Python pandas numpy scipy scikit learn matplotlib Sql Java JavaScript JQuery Machine learn Regresion SVM Bayes Random Forest Decision Tres Bosting techniques Cluster Analysis Word Embeding Sentiment Analysis Natural Language procesing Dimensionality reduction Topic Modeling LDA NMF PCA Neural Nets Database Visualizations Mysql SqlServer Casandra Hbase ElasticSearch Plotly kibana matplotlib gplot Tableau Others Regular Expresion HTML Angular Logstash Kafka Python Flask Git Docker computer vision Open understand Dep learn Education Details Data Science Asurance AsociateData Science Asurance Asociate Ernst Young LPSkil Details JAVASCRIPT Exprience months jQuery Exprience months Python Exprience monthsCompany Details company Ernst Young description Fraud Investigations Dispute Services Asurance TECHNOLOGY ASISTED REVIEW TAR Technology Asisted Review asists acelerating review proces run analytics generate report Core member team help develop automate review platform tol scratch asisting discovery domain tol implement predictive cod topic model automate review result reduce labor cost time spend lawyers review Understand end end flow solution research development clasification model predictive analysis mine information present text data Worked analyze output precision monitor entire tol TAR asists predictive cod topic model evidence folowing standards Developed clasifier model order identify red flag fraud relate isues Tols Technologies Python scikit learn tfidf word vec doc vec cosine similarity Bayes LDA NMF topic model Vader text blob sentiment analysis Matplot lib Tableau dashboard report MULTIPLE DATA SCIENCE AND ANALYTIC PROJECTS USA CLIENTS TEXT ANALYTICS MOTOR VEHICLE CUSTOMER REVIEW DATA Received customer fedback survey data past one year Performed sentiment Positive Negative Neutral time series analysis customer coments acros categories Created heat map term survey category base frequency word Extracted Positive Negative word acros Survey categories plot Word cloud Created customize tableau dashboards efective report visualizations CHATBOT Developed user friendly chatbot one Products handle simple question hours operation reservation options This chat bot serve entire product relate question Giving overview tol via platform also give recomendation responses user question build chain relevant answer This inteligence build pipeline question per user requirement ask relevant recomended question Tols Technologies Python Natural language procesing NLTK spacy topic model Sentiment analysis Word Embeding scikit learn JavaScript JQuery SqlServer INFORMATION GOVERNANCE Organizations make inform decisions information store The integrate Information Governance portfolio synthesize inteligence acros unstructured data source facilitate action ensure organizations best position counter information risk Scan data multiple source format parse diferent file format extract Meta data information push result index elastic search create customize interactive dashboards use kibana Preforming ROT Analysis data give information data help identify content either Redundant Outdated Trivial Preforming ful text search analysis elastic search predefined methods tag personaly identifiable information social security number adreses name etc frequently target cyber atacks Tols Technologies Python Flask Elastic Search Kibana FRAUD ANALYTIC PLATFORM Fraud Analytics investigative platform review red flag case FAP Fraud Analytics investigative platform inbuilt case manager suite Analytics various ERP systems use clients interogate Acounting systems identify anomalies indicators fraud run advance analytics Tols Technologies HTML JavaScript SqlServer JQuery Botstrap Node
1,Education Details May May UIT RGPVData ScientistData Scientist MatelabsSkil Details Python Exprience Les year months Statsmodels Exprience months AWS Exprience Les year months Machine learn Exprience Les year months Sklearn Exprience Les year months Scipy E

In [25]:
data.tail(2) # data after cleaning

,TEXT
1255,MAJOR CONRAD PREDOM Fairchild Suite USAF Academy conrad predom usafa edu TEACHING EXPERIENCE Asistant Profesor Management Director Curiculum Air Force Academy Present Cordinated clases anualy faculty members cadets develop curiculum advance management course provide clasrom instruction section cadets semesters multiple uper level course provide focus profesional mentorship cadets Diamond Star Instructor Pilot Air Force Academy Present Performed Instructor Pilot duties single engine Diamond Star trainer aircraft provide instruction season Air Force pilot wel Air Force Academy cadets cockpit Cesna Instructor Pilot Laughlin Air Force Base Performed Instructor Pilot duties Air Force largest pilot train base Air Force primary jet trainer provide clasrom instruction flight instruction aspects fly include trafic paterns land aerobatic flight spin train instrument fly formation fly day night cros country navigation Schweitzer Instructor Pilot Air Force Academy Performed Instructor Pilot duties Air Force Academy Soar program provide flight line sailplane cockpit instruction semesters cadets focus aero tow primary maneuver trafic paterns land FLYING EXPERIENCE Present Flight Hours Instructor Pilot Hours Diamond Star USAF Hours Flight Lead Hours McDonel Douglas Strike Eagle Hours Northrop Flight Hours Instructor Pilot Hours Cesna Flight Hours Northrop Flight Hours Instructor Pilot Hours Schweitzer USAF Other Aircraft Flown include Slingsby Firefly Cesna Cesna USAF DUTY HISTORY Present Asistant Profesor Management Director Curiculum Air Force Academy Director Staf Seventh Air Force Operations Plans Osan Air Base Republic Korea Comander Plans Flight Multi ship Flight Lead Seymour Johnson Air Force Base Student Pilot Seymour Johnson Air Force Base Executive Oficer Operations Group Instructor Pilot Laughlin Air Force Base Air Force Pilot Training Student Laughlin Air Force Base FORMAL EDUCATION MBA Busines Administration Management University Pitsburgh GPA Operations Research Air Force Academy GPA PROFESIONAL MILITARY EDUCATION Air Comand Staf Colege Squadron Oficer Schol RATINGS AND CERTIFICATIONS Certified Flight Instructor Federal Aviation Administration Senior Pilot United States Air Force Comercial Pilot Federal Aviation Administration Pilot United States Air Force Basic Parachutist United States Air Force cHONOR AWARDS Field Grade Oficer Quarter Division Level Dept Management Air Force Academy Outstanding Academy Educator Award Dept Management Air Force Academy Field Grade Oficer Quarter Division Level Dept Management Air Force Academy Theodore Helmer Teaching Award Department Management Air Force Academy Company Grade Oficer Quarter Operations Suport Squadron Seymour Johnson Air Force Base Leadership Award Fighter Squadron Seymour Johnson Air Force Base Instructor Pilot Year Flying Training Squadron Laughlin Air Force Base Company Grade Oficer Quarter Flying Training Squadron Laughlin Air Force Base Beta Gama Sigma Honor Society Katz Graduate Schol Busines University Pitsburgh Distinguished Graduate United States Air Force Academy Omega Rho Honor Society Operations Research United States Air Force Academy AIR FORCE AWARDS AND DECORATIONS Meritorious Service Medal Air Force Comendation Medal Air Force Achievement Medal Air Force Outstanding Unit Award Air Force Organizational Excelence Award Combat Readines Medal National Defense Service Medal Global War Terorism Service Medal Korean Defense Service Medal Longevity Service Smal Arms Expert Marksmanship Ribon Pistol Training Ribon
1256,STACY SAMPLE cel email com Qualifications Flight Time Flight Ratings ATP Airline Transport Pilot CFI Rotorcraft Helicopter Type rat Medical Clas Experience Sikorsky Test Pilot Jul Present Total Pilot Comand Pilot Helicopter Multi Engine Cros Country Instrument IFR Hod Actual Weather Night Night Vision Gogle Ofshore Overwater External Load EMS Mountain Sikorsky Aircraft Corporation Coatesvile Production Test Pilot Maintenance test pilot engineri

In [26]:
data.sample(2) # data after cleaning

,TEXT
829,Art Editor Print Magazine Resume Template Mariane Lawrence Abe Detroit Michigan Phone lawrence ycmail com Objective work Art Editor large circulation magazine specialize photography visual arts hobies Sumary Qualification years experience print media profesion last years head art department large hoby magazine Excelent creative artistic skils Excelent comand Adobe Photoshop PageMaker edit tols Carer Experience Job History Present Asociate Art Editor Digital Hobies Americas Detroit Conduct regular editorial staf metings artists photographers layout artists brainstorm ideas tactical loks upcoming edition along line overal thematic design publication Simulate artistic layout PageMaker aproval Editor Chief Atend seminars lecture technological advance print media Develop web version magazine cordination web artists Cordinate webmaster ensure website miror print version publication Colaborate layout artist finalize edition Ensure graphics photo materials submit days copy deadline leave enough time finalize selection Education MBA Michigan State University Comercial Industrial Arts Milwauke Institute Art Design Profesional reference wil furnish upon request
1079,Acount Manager Dayjob Ltd Vyse Stret Birmingham info dayjob com Personal statement Multi skiled profesional creative mindset talkative personality hand aproach problem solve Peter ability listen gain trust clients also bring closure action project resolution sale customers pose operational sales experience manage pipeline oportunities acurately forecast revenue conduct acount forecast review senior management Right ken find chalenging position within ambitious organisation wil able continue increase work experience develop abilities Employment history Distribution Company Birmingham ACOUNT MANAGER April Present Esentialy responsible manage key acounts maintain long term relationship acounts maximise sales oportunities within Also charge implementation sales strategies manage sales profitability overal busines relationship key acounts Duties Identify implement proces improvements cordinating activities betwen departments Advising change trend marketplace activities competitors Using contact generate new busines Ensuring clients understand service products ofer Establish strong client relationships regular close contact Asisting Marketing Executive content company literature go corespondence Constructing maintain Campaign Plans Manufacturing Company West Bromwich ACOUNT MANAGER October April Peter McDonald Insurance Company Dudley SALES EXECUTIVE July October Areas expertise Administration Analyse sales data Writing busines plan Chairing metings Acount management Social Media market Public speak Event management Product launch Report write Budget administration Promotions Profesional skils Know analyse result identify explain variance Analysing interpret present result Sound understand comercial proceses Prepare distribute information company clients Ability understand new products significance impact market place Answer clients question ensure problems resolve conduct folow task Able understand clients neds quickly Strong listen question skils Academic qualifications Birmingham North University Busines Management Hons City Guilds Sales Management Birmingham South Colege Busines Studies Diploma References Available request cCopyright information Please read This Acount Manager template copyright Dayjob Ltd Job sekers may download use particular example personal use help write one You also welcome link page site dayjob com However template must distribute make available websites without prior permision For question relate use template please email info dayjob com


In [27]:
data.shape

(1255, 1)

### Remove Domain Specific Stopwords

In [28]:
freq_words = get_freq_word(list(data.TEXT))

In [29]:
freq_words.columns = ['WORD', 'COUNT']
freq_words.head(20)

,WORD,COUNT
1309,University,3432
541,project,2400
480,work,2330
290,use,2324
602,include,2097
260,The,2065
522,Management,1941
105,team,1878
814,experience,1874
933,skils,1866


In [30]:
freq_words.to_csv('freq_words_3.csv')

after take a decision about what is our domain specific word, we will start remove them from the dataset

In [31]:
words_df = pd.read_csv('freq_words_after_delete_col_3.csv')

In [32]:
words_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [33]:
words_df.fillna(0, inplace = True)

In [34]:
domain_stop_words_df = words_df[words_df.DELETE == 1]

In [35]:
domain_stop_words_df.drop('DELETE', axis = 1, inplace = True)

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [36]:
domain_stop_words_df

,WORD,COUNT
0,University,3432
1,project,2400
2,work,2330
3,use,2324
4,include,2097
...,...,...
393,day,283
394,specific,282
395,Certificate,282
396,Dawson,281


In [37]:
data.index = range(0, len(data))

In [38]:
df_after = remove_domain_stop_words(list(domain_stop_words_df.WORD), data)

In [39]:
df_after.head(2)

,TEXT
0,Programing Python pandas numpy scipy scikit learn matplotlib Sql Java JavaScript JQuery Machine learn Regresion SVM Bayes Random Forest Decision Tres Bosting techniques Cluster Analysis Word Embeding Sentiment Analysis Natural Language procesing Dimensionality reduction Topic Modeling LDA NMF PCA Neural Nets Database Visualizations Mysql SqlServer Casandra Hbase ElasticSearch Plotly kibana matplotlib gplot Tableau Others Regular Expresion Angular Logstash Kafka Python Flask Git Docker computer vision Open Dep learn Data Science Asurance AsociateData Science Asurance Asociate Ernst Young LPSkil JAVASCRIPT jQuery Python monthsCompany Ernst Young Fraud Investigations Dispute Services Asurance TECHNOLOGY ASISTED REVIEW TAR Technology asists acelerating run analytics generate Core develop automate platform tol scratch asisting discovery domain tol predictive cod topic automate reduce labor cost spend lawyers Understand flow solution research development clasification predictive analysis mine information text data analyze output precision monitor entire tol TAR asists predictive cod topic evidence folowing standards Developed clasifier red flag fraud Tols Technologies Python scikit learn tfidf word vec doc vec cosine similarity Bayes LDA NMF topic Vader text blob sentiment analysis Matplot lib Tableau dashboard MULTIPLE DATA SCIENCE ANALYTIC PROJECTS USA CLIENTS TEXT ANALYTICS MOTOR VEHICLE CUSTOMER REVIEW DATA Received customer fedback survey data past sentiment Positive Negative Neutral series analysis customer coments acros categories Created heat map term survey category frequency word Extracted Positive Negative word acros Survey categories plot Word cloud Created customize tableau dashboards visualizations CHATBOT Developed friendly chatbot Products simple question hours operation reservation options chat bot serve entire product question Giving overview tol via platform give recomendation responses question chain relevant answer inteligence pipeline question requirement ask relevant recomended question Tols Technologies Python Natural language procesing NLTK spacy topic Sentiment analysis Word Embeding scikit learn JavaScript JQuery SqlServer INFORMATION GOVERNANCE Organizations inform decisions information store integrate Information Governance portfolio synthesize inteligence acros unstructured data source facilitate action organizations counter information risk Scan data multiple source format parse format extract Meta data information push index elastic search customize interactive dashboards kibana Preforming ROT Analysis data give information data either Redundant Outdated Trivial Preforming ful text search analysis elastic search predefined methods tag personaly identifiable information security number adreses frequently target cyber atacks Tols Technologies Python Flask Elastic Search Kibana FRAUD ANALYTIC PLATFORM Fraud Analytics investigative platform red flag FAP Fraud Analytics investigative platform inbuilt manager suite Analytics ERP systems clients interogate Acounting systems anomalies indicators fraud run advance analytics Tols Technologies JavaScript SqlServer JQuery Botstrap Node
1,UIT RGPVData ScientistData Scientist MatelabsSkil Python Les Statsmodels AWS Les Machine learn Les Sklearn Les Scipy Les Keras Les monthsCompany Matelabs Platform busines profesionals dumies enthusiasts Koramangala block Achievements Tasks behind sukh sagar Bengaluru Developed deploy auto preprocesing step machine learn mainly mising value treatment outlier detection encode scale feature selection dimensionality reduction Deployed automate clasification regresion linkedin aditya rathore Reasearch deploy series forecast ARIMA SARIMAX Holt winter Prophet meta feature extract problem github rathorology Implemented art research outlier detection mix atributes Matelabs


In [40]:
df_after.tail(2)

,TEXT
1253,MAJOR CONRAD PREDOM Fairchild Suite USAF Academy conrad predom usafa TEACHING Asistant Management Director Curiculum Air Force Academy Cordinated clases anualy faculty cadets develop curiculum advance management clasrom instruction cadets semesters multiple uper mentorship cadets Diamond Star Instructor Pilot Air Force Academy Instructor Pilot duties single engine Diamond Star trainer aircraft instruction season Air Force pilot Air Force Academy cadets cockpit Cesna Instructor Pilot Laughlin Air Force Base Instructor Pilot duties Air Force largest pilot Air Force primary jet trainer clasrom instruction flight instruction aspects fly trafic paterns land aerobatic flight spin instrument fly formation fly night cros country navigation Schweitzer Instructor Pilot Air Force Academy Instructor Pilot duties Air Force Academy Soar program flight sailplane cockpit instruction semesters cadets aero tow primary maneuver trafic paterns land FLYING Flight Hours Instructor Pilot Hours Diamond Star USAF Hours Flight Lead Hours McDonel Douglas Strike Eagle Hours Northrop Flight Hours Instructor Pilot Hours Cesna Flight Hours Northrop Flight Hours Instructor Pilot Hours Schweitzer USAF Other Aircraft Flown Slingsby Firefly Cesna Cesna USAF DUTY HISTORY Asistant Management Director Curiculum Air Force Academy Director Staf Seventh Air Force Operations Plans Osan Air Base Republic Korea Comander Plans Flight Multi ship Flight Lead Seymour Johnson Air Force Base Student Pilot Seymour Johnson Air Force Base Executive Oficer Operations Instructor Pilot Laughlin Air Force Base Air Force Pilot Student Laughlin Air Force Base FORMAL MBA Busines Administration Management Pitsburgh Operations Research Air Force Academy PROFESIONAL MILITARY Air Comand Staf Squadron Oficer RATINGS CERTIFICATIONS Certified Flight Instructor Federal Aviation Administration Pilot United States Air Force Comercial Pilot Federal Aviation Administration Pilot United States Air Force Basic Parachutist United States Air Force cHONOR AWARDS Field Grade Oficer Quarter Division Level Dept Management Air Force Academy Outstanding Academy Educator Dept Management Air Force Academy Field Grade Oficer Quarter Division Level Dept Management Air Force Academy Theodore Helmer Teaching Management Air Force Academy Grade Oficer Quarter Operations Suport Squadron Seymour Johnson Air Force Base Leadership Fighter Squadron Seymour Johnson Air Force Base Instructor Pilot Year Flying Squadron Laughlin Air Force Base Grade Oficer Quarter Flying Squadron Laughlin Air Force Base Beta Gama Sigma Honor Katz Busines Pitsburgh Distinguished United States Air Force Academy Omega Rho Honor Operations Research United States Air Force Academy AIR FORCE AWARDS DECORATIONS Meritorious Service Medal Air Force Comendation Medal Air Force Achievement Medal Air Force Outstanding Unit Air Force Organizational Excelence Combat Readines Medal Defense Service Medal Global War Terorism Service Medal Korean Defense Service Medal Longevity Service Smal Arms Expert Marksmanship Ribon Pistol Ribon
1254,STACY SAMPLE cel Qualifications Flight Time Flight Ratings ATP Airline Transport Pilot CFI Rotorcraft Helicopter Type rat Medical Clas Sikorsky Test Pilot Jul Total Pilot Comand Pilot Helicopter Multi Engine Cros Country Instrument IFR Hod Actual Weather Night Night Vision Gogle Ofshore Overwater External Load EMS Mountain Sikorsky Aircraft Corporation Coatesvile Production Test Pilot Maintenance test pilot enginering test flight civil foreign military flight instructor customer demonstrations international IFR VFR overwater Bel Corporate IFR Captain Jan Jun Elite Aviation Los Angeles Single Pilot Bel Corporate IFR Captain airport land logistics air grind security ocasional Scren Actors Guild motion picture Part Sikorsky EMS Captain Oct Dec Children Hospital Los Angeles Helinet Aviation Van Nuys Single Pilot EMS Captain Mountainous roftop confine areas charter transport film ENG maintenance test flight Part Chart

In [41]:
df_after.sample(2)

TEXT
797  JIL JACK Main   JilJack gmail  JilJack gplus JilJack MULTIMEDIA MANAGER COMUNICATIONS STRATEGIST CONTENT DEVELOPER Market savy writer editor  produce profitable  video audio ezine  cJil combine flair creative intuitive  market trend consumer  unbeatable technical  edit management  never know anyone evaluate landscape design strategy execute   Jil Peter Pan Overseing Editor XYX  View endorsements via LinkedIn  Achievements Managing Web Editor XYZ    Produced  edit video text website   news promos headline teasers newsleters blog product descriptions Known  cgo guy creative ideas sugestions Envisioned execute fresh creative market promotion   audio blog Internet magazine video biography  Ratings viewer loyalty grow improve  external rank profitability  web analytics market research customer analysis drive website enhancements  feature Production Manager DEF Satelite Radio  Analyzed  trafic  Gogle analytics Lyris Agency Edition SiteCatalyst Identified strategic paterns trend popular  recomended      view thre  Supervised  overhaul compliance   Conducted  asesment  market oportunities  test design stimulate comerce  profitability Authored  Web  newsleters   podcast highlight  mision Management atributed  membership rat highquality  audio  Supervising Web Editor ABC     HIGHLIGHTS   Produced  six hours  radio   original show Repackaged versions popular program  station quickly become favorite satelite radio target Gen audience Raised station profile   enginering exclusive live studio performances  profile artists Coldplay INXS David Gray dozens Writing Podcasts Write design podcast download peak trafic unique visit month Podcasts feature morning drive KABC Frelance writer podcaster technical editorial blog Increase Revenues XYZ outline sales  rise Visitors spend  XYZ  fuel  renewals baner ads drive prospect  aded thousands  revenue   Arts Media Studies Journalism Quens  Flushing Computer  Pro Tols Col Edit Pro Sony Sound Forge Photoshop Final Cut Pro Serena Colage Content Management System                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

### Store The Cleaned Dataset Into Pickles

In [42]:
write_pickle('df_2_after_cleaning.pickle', df_after)

### Add SPACY_TEXT Colum

In [43]:
nlp = spacy.load('en_core_web_sm')
df_after['SPACY_TEXT'] = list(nlp.pipe(df_after.TEXT))

In [44]:
df_after.head(2)

,TEXT,SPACY_TEXT
0,Programing Python pandas numpy scipy scikit learn matplotlib Sql Java JavaScript JQuery Machine learn Regresion SVM Bayes Random Forest Decision Tres Bosting techniques Cluster Analysis Word Embeding Sentiment Analysis Natural Language procesing Dimensionality reduction Topic Modeling LDA NMF PCA Neural Nets Database Visualizations Mysql SqlServer Casandra Hbase ElasticSearch Plotly kibana matplotlib gplot Tableau Others Regular Expresion Angular Logstash Kafka Python Flask Git Docker computer vision Open Dep learn Data Science Asurance AsociateData Science Asurance Asociate Ernst Young LPSkil JAVASCRIPT jQuery Python monthsCompany Ernst Young Fraud Investigations Dispute Services Asurance TECHNOLOGY ASISTED REVIEW TAR Technology asists acelerating run analytics generate Core develop automate platform tol scratch asisting discovery domain tol predictive cod topic automate reduce labor cost spend lawyers Understand flow solution research development clasification predictive analysis mine information text data analyze output precision monitor entire tol TAR asists predictive cod topic evidence folowing standards Developed clasifier red flag fraud Tols Technologies Python scikit learn tfidf word vec doc vec cosine similarity Bayes LDA NMF topic Vader text blob sentiment analysis Matplot lib Tableau dashboard MULTIPLE DATA SCIENCE ANALYTIC PROJECTS USA CLIENTS TEXT ANALYTICS MOTOR VEHICLE CUSTOMER REVIEW DATA Received customer fedback survey data past sentiment Positive Negative Neutral series analysis customer coments acros categories Created heat map term survey category frequency word Extracted Positive Negative word acros Survey categories plot Word cloud Created customize tableau dashboards visualizations CHATBOT Developed friendly chatbot Products simple question hours operation reservation options chat bot serve entire product question Giving overview tol via platform give recomendation responses question chain relevant answer inteligence pipeline question requirement ask relevant recomended question Tols Technologies Python Natural language procesing NLTK spacy topic Sentiment analysis Word Embeding scikit learn JavaScript JQuery SqlServer INFORMATION GOVERNANCE Organizations inform decisions information store integrate Information Governance portfolio synthesize inteligence acros unstructured data source facilitate action organizations counter information risk Scan data multiple source format parse format extract Meta data information push index elastic search customize interactive dashboards kibana Preforming ROT Analysis data give information data either Redundant Outdated Trivial Preforming ful text search analysis elastic search predefined methods tag personaly identifiable information security number adreses frequently target cyber atacks Tols Technologies Python Flask Elastic Search Kibana FRAUD ANALYTIC PLATFORM Fraud Analytics investigative platform red flag FAP Fraud Analytics investigative platform inbuilt manager suite Analytics ERP systems clients interogate Acounting systems anomalies indicators fraud run advance analytics Tols Technologies JavaScript SqlServer JQuery Botstrap Node,"( , Programing, , Python, pandas, numpy, scipy, scikit, learn, matplotlib, Sql, Java, JavaScript, JQuery, Machine, learn, Regresion, SVM, Bayes, Random, Forest, Decision, Tres, Bosting, techniques, Cluster, Analysis, Word, Embeding, Sentiment, Analysis, Natural, Language, procesing, Dimensionality, reduction, Topic, Modeling, LDA, NMF, PCA, Neural, Nets, Database, Visualizations, Mysql, SqlServer, Casandra, Hbase, ElasticSearch, Plotly, kibana, matplotlib, gplot, Tableau, Others, Regular, Expresion, , Angular, Logstash, Kafka, Python, Flask, Git, Docker, computer, vision, Open, , Dep, learn, , Data, Science, Asurance, AsociateData, Science, Asurance, Asociate, Ernst, Young, LPSkil, , JAVASCRIPT, , jQuery, , Python, , monthsCompany, , Ernst, Young, , Fraud, Investigations, Dispute, Services, Asurance, ...)"
1,UIT

In [45]:
df_after.tail(2)

,TEXT,SPACY_TEXT
1253,MAJOR CONRAD PREDOM Fairchild Suite USAF Academy conrad predom usafa TEACHING Asistant Management Director Curiculum Air Force Academy Cordinated clases anualy faculty cadets develop curiculum advance management clasrom instruction cadets semesters multiple uper mentorship cadets Diamond Star Instructor Pilot Air Force Academy Instructor Pilot duties single engine Diamond Star trainer aircraft instruction season Air Force pilot Air Force Academy cadets cockpit Cesna Instructor Pilot Laughlin Air Force Base Instructor Pilot duties Air Force largest pilot Air Force primary jet trainer clasrom instruction flight instruction aspects fly trafic paterns land aerobatic flight spin instrument fly formation fly night cros country navigation Schweitzer Instructor Pilot Air Force Academy Instructor Pilot duties Air Force Academy Soar program flight sailplane cockpit instruction semesters cadets aero tow primary maneuver trafic paterns land FLYING Flight Hours Instructor Pilot Hours Diamond Star USAF Hours Flight Lead Hours McDonel Douglas Strike Eagle Hours Northrop Flight Hours Instructor Pilot Hours Cesna Flight Hours Northrop Flight Hours Instructor Pilot Hours Schweitzer USAF Other Aircraft Flown Slingsby Firefly Cesna Cesna USAF DUTY HISTORY Asistant Management Director Curiculum Air Force Academy Director Staf Seventh Air Force Operations Plans Osan Air Base Republic Korea Comander Plans Flight Multi ship Flight Lead Seymour Johnson Air Force Base Student Pilot Seymour Johnson Air Force Base Executive Oficer Operations Instructor Pilot Laughlin Air Force Base Air Force Pilot Student Laughlin Air Force Base FORMAL MBA Busines Administration Management Pitsburgh Operations Research Air Force Academy PROFESIONAL MILITARY Air Comand Staf Squadron Oficer RATINGS CERTIFICATIONS Certified Flight Instructor Federal Aviation Administration Pilot United States Air Force Comercial Pilot Federal Aviation Administration Pilot United States Air Force Basic Parachutist United States Air Force cHONOR AWARDS Field Grade Oficer Quarter Division Level Dept Management Air Force Academy Outstanding Academy Educator Dept Management Air Force Academy Field Grade Oficer Quarter Division Level Dept Management Air Force Academy Theodore Helmer Teaching Management Air Force Academy Grade Oficer Quarter Operations Suport Squadron Seymour Johnson Air Force Base Leadership Fighter Squadron Seymour Johnson Air Force Base Instructor Pilot Year Flying Squadron Laughlin Air Force Base Grade Oficer Quarter Flying Squadron Laughlin Air Force Base Beta Gama Sigma Honor Katz Busines Pitsburgh Distinguished United States Air Force Academy Omega Rho Honor Operations Research United States Air Force Academy AIR FORCE AWARDS DECORATIONS Meritorious Service Medal Air Force Comendation Medal Air Force Achievement Medal Air Force Outstanding Unit Air Force Organizational Excelence Combat Readines Medal Defense Service Medal Global War Terorism Service Medal Korean Defense Service Medal Longevity Service Smal Arms Expert Marksmanship Ribon Pistol Ribon,"(MAJOR, CONRAD, PREDOM, Fairchild, Suite, USAF, Academy, conrad, predom, usafa, , TEACHING, , Asistant, , Management, Director, Curiculum, Air, Force, Academy, , Cordinated, clases, anualy, faculty, , cadets, develop, curiculum, advance, management, , clasrom, instruction, , cadets, semesters, multiple, uper, , mentorship, cadets, Diamond, Star, Instructor, Pilot, Air, Force, Academy, , Instructor, Pilot, duties, single, engine, Diamond, Star, trainer, aircraft, , instruction, season, Air, Force, pilot, , Air, Force, Academy, cadets, cockpit, Cesna, Instructor, Pilot, Laughlin, Air, Force, Base, , Instructor, Pilot, duties, Air, Force, largest, pilot, , Air, Force, primary, jet, trainer, , clasrom, instruction, flight, instruction, aspects, fly, ...)"
1254,STACY SAMPLE cel Qualifications Flight Time Flight Ratings ATP Airline Transport Pilot CFI Rotorcraft Helicopter Type rat Medical Clas Sikorsky Test Pil

In [46]:
df_after.sample(2)

,TEXT,SPACY_TEXT
751,Inside Sales Template Jason Thompson Wild Maryland gmail Forte Poses extensive consumer behavior negotiate Proficient cold caling telesales Adept sales Skiled computer software aplication analytical player Proficient multitasking King Sales til Inside Sales Rep Major start close sales promotions products Price negotiations clients customers client query generate analyze give presentations update clients Amigo Sales Sales Sales Rep Major sales promotions products Price negotiations clients customers client query Educational Qualification degre Arts Diploma Management know Fluent English Latin Other Interests Cary display strategy products store Analyze technological gadgets Significant Achievement Won trip Paris exceptionaly,"(Inside, Sales, , Template, Jason, Thompson, Wild, , Maryland, gmail, , Forte, Poses, extensive, , consumer, behavior, , negotiate, , Proficient, cold, caling, telesales, Adept, sales, Skiled, computer, software, aplication, , analytical, , player, Proficient, multitasking, , King, Sales, til, , Inside, Sales, Rep, , Major, , start, close, sales, promotions, products, Price, negotiations, clients, customers, , client, query, , generate, analyze, , give, presentations, update, clients, Amigo, Sales, Sales, Sales, Rep, , Major, , sales, promotions, products, Price, negotiations, clients, customers, , client, query, Educational, Qualification, , degre, Arts, , Diploma, Management, , know, Fluent, English, Latin, Other, Interests, ...)"
1015,Enginering Fresher,"(Enginering, Fresher, )"


In [47]:
# Delete any Entities 
for i in range(0, len(df_after)):
    ents = df_after.SPACY_TEXT[i]
    j = 0
    for ent in ents.ents:
        pattern = r'\b' + ent.text + r'\b'
        new_text = re.sub(pattern,' ', df_after.TEXT[j])
        df_after.TEXT[j] = new_text
        j += 1

In [48]:
df_after.TEXT = df_after.TEXT.apply(lambda x: x.lower())

In [49]:
df_after.head(2)

,TEXT,SPACY_TEXT
0,programing python pandas numpy scipy scikit learn matplotlib javascript jquery machine learn regresion svm bayes random forest decision tres bosting techniques cluster analysis word embeding sentiment analysis natural language procesing dimensionality reduction topic modeling lda nmf pca neural nets database visualizations mysql sqlserver casandra hbase elasticsearch plotly kibana matplotlib gplot others regular expresion angular logstash kafka python flask git docker computer vision open dep learn data science asurance asociatedata science asurance asociate ernst young lpskil javascript jquery python monthscompany ernst young fraud investigations dispute services asurance technology asisted review tar technology asists acelerating run analytics generate develop automate platform tol scratch asisting discovery domain tol predictive cod topic automate reduce labor cost spend lawyers understand flow solution research development clasification predictive analysis mine information text data analyze output precision monitor entire tol tar asists predictive cod topic evidence folowing standards developed clasifier red flag fraud tols technologies python scikit learn tfidf word vec doc vec cosine similarity bayes lda nmf topic vader text blob sentiment analysis matplot lib dashboard multiple data science analytic projects usa clients text analytics motor vehicle customer review data received customer fedback survey data past sentiment positive negative neutral series analysis customer coments acros categories created heat map term survey category frequency word extracted positive negative word acros survey categories plot word cloud created customize tableau dashboards visualizations chatbot developed friendly chatbot products simple question operation reservation options chat bot serve entire product question giving overview tol via platform give recomendation responses question chain relevant answer inteligence pipeline question requirement ask relevant recomended question tols technologies python natural language procesing nltk spacy topic sentiment analysis word embeding scikit learn javascript jquery sqlserver information governance organizations inform decisions information store integrate information governance portfolio synthesize inteligence acros unstructured data source facilitate action organizations counter information risk scan data multiple source format parse format extract meta data information push index elastic search customize interactive dashboards kibana preforming rot analysis data give information data either redundant outdated trivial preforming ful text search analysis elastic search predefined methods tag personaly identifiable information security number adreses frequently target cyber atacks tols technologies python flask elastic search kibana fraud analytic platform fraud analytics investigative platform red flag fap fraud analytics investigative platform inbuilt manager suite analytics systems clients interogate acounting systems anomalies indicators fraud run advance analytics tols technologies javascript sqlserver jquery botstrap node,"( , Programing, , Python, pandas, numpy, scipy, scikit, learn, matplotlib, Sql, Java, JavaScript, JQuery, Machine, learn, Regresion, SVM, Bayes, Random, Forest, Decision, Tres, Bosting, techniques, Cluster, Analysis, Word, Embeding, Sentiment, Analysis, Natural, Language, procesing, Dimensionality, reduction, Topic, Modeling, LDA, NMF, PCA, Neural, Nets, Database, Visualizations, Mysql, SqlServer, Casandra, Hbase, ElasticSearch, Plotly, kibana, matplotlib, gplot, Tableau, Others, Regular, Expresion, , Angular, Logstash, Kafka, Python, Flask, Git, Docker, computer, vision, Open, , Dep, learn, , Data, Science, Asurance, AsociateData, Science, Asurance, Asociate, Ernst, Young, LPSkil, , JAVASCRIPT, , jQuery, , Python, , monthsCompany, , Ernst, Young, , Fraud, Investigations, Dispute, Services, Asurance, ...)"
1,uit rgpvdata scientistdata scientist matela

### Topic Modeling and Removing Domain Specific Stop Words Iterations

In [50]:
vectorizer = CountVectorizer(max_features=20000,
                             stop_words='english', token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True)

doc_word = vectorizer.fit_transform(df_after.TEXT)
words = list(np.asarray(vectorizer.get_feature_names()))

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [51]:
topic_model = ct.Corex(n_hidden=6, words=words, seed=1)
topic_model.fit(doc_word, words=words, docs=df_after.TEXT)

In [52]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))
topics

0: use,services,students,public,human,future,design,integrated,introduction,conference
1: development,strategic,develop,manage,analysis,manager,improve,managed,management,program
2: publications,awards,curiculum,vitae,cthe,undergraduate,study,guide,network,journal
3: comunity,asociation,teaching,student,research,council,teach,chair,faculty,asistant
4: permision,welcome,link,read,distribute,dayjob,question,areas,ccopyright,prior
5: birth,date,marital,status,indian,hindi,nationality,maried,male,known


[[('use', 0.09372323527532923, 1.0),
  ('services', 0.08145305109485508, 1.0),
  ('students', 0.07705668312409143, 1.0),
  ('public', 0.07561446907178, 1.0),
  ('human', 0.07552755559965742, 1.0),
  ('future', 0.07136567675090484, 1.0),
  ('design', 0.06802289605988017, 1.0),
  ('integrated', 0.06756481048229936, 1.0),
  ('introduction', 0.06268468390386267, 1.0),
  ('conference', 0.0626178684519003, 1.0)],
 [('development', 0.14830911140605868, 1.0),
  ('strategic', 0.12744691181035464, 1.0),
  ('develop', 0.11199512101640648, 1.0),
  ('manage', 0.11099333686745345, 1.0),
  ('analysis', 0.11097253203416431, 1.0),
  ('manager', 0.11003105897331929, 1.0),
  ('improve', 0.10717319626970236, 1.0),
  ('managed', 0.10316298965938438, 1.0),
  ('management', 0.1029704004193532, 1.0),
  ('program', 0.10252860579214802, 1.0)],
 [('publications', 0.08678713563589584, 1.0),
  ('awards', 0.07530296442898717, 1.0),
  ('curiculum', 0.07382621771941432, 1.0),
  ('vitae', 0.07061159418550146, 1.0),
  

In [53]:
tfidf = TfidfVectorizer(stop_words='english')
doc_words = tfidf.fit_transform(df_after.TEXT)
tfidf_df = pd.DataFrame(doc_words.toarray(),columns=tfidf.get_feature_names())

In [54]:
topic_nums = list(np.arange(3, 11, 1))
for num in topic_nums:
    nmf = NMF(num)
    nmf.fit_transform(doc_words)
    display_topics(nmf, tfidf.get_feature_names(), 100)
    print("------------------------------------------")

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
design, enginering, web, software, test, development, management, java, data, program, systems, sql, developer, technical, computer, busines, database, aplication, technology, science, enginer, develop, information, server, research, suport, aplications, technologies, windows, service, net, client, health, law, maharashtra, environment, oracle, analysis, activities, designer, tols, php, network, microsoft, mysql, les, javascript, quality, civil, role, developed, mechanical, pune, production, manage, linux, student, work, visual, pvt, electrical, learn, asistant, maintain, clients, relevant, product, users, testing, functional, medical, programing, implementation, graphic, website, title, code, script, human, academic, asociation, automation, manager, python, maintenance, created, jquery, services, databases, enginers, xml, control, customer, language, performance, teach, adobe, equipment, art, android

Topic  1
sales, market, customer, busines, marketing, product, customers, 

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
web, java, software, design, test, sql, developer, data, database, development, server, aplication, enginering, net, systems, oracle, technologies, maharashtra, aplications, busines, technical, windows, mysql, les, php, javascript, computer, technology, pune, tols, jquery, linux, client, pvt, testing, enginer, script, management, develop, users, automation, python, environment, xml, analysis, science, android, databases, visual, suport, functional, code, ajax, role, asp, network, programing, designer, description, flash, requirement, jsp, monthscompany, website, product, platform, servers, program, microsoft, hibernate, unix, operating, framework, integration, information, mumbai, implementation, architecture, used, configuration, analyst, core, studio, user, mvc, solution, sap, tol, mobile, agile, developed, documentation, tech, custom, interface, source, adobe, manage, object, maintenance

Topic  1
sales, market, customer, busines, product, marketing, customers, manager, pr

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


...


Topic  0
management, busines, human, program, development, resources, manager, operations, cost, manage, performance, recruitment, administration, service, quality, health, strategic, develop, market, analysis, policies, resource, managed, budget, executive, risk, compensation, relations, proceses, suport, managers, benefit, implementation, personel, payrol, systems, insurance, strategy, employe, hire, client, organizational, developed, delivery, improve, compliance, division, data, safety, consultant, candidates, growth, senior, maintain, activities, revenue, medical, services, salary, global, reduce, ontario, healthcare, functional, information, profesional, vendor, eficiency, acros, achieve, monitor, initiatives, policy, test, value, technology, save, enterprise, operational, control, sucesfuly, technical, los, retention, customer, drive, benefits, stafing, talent, clients, product, deliver, sucesful, director, proces, labor, acomplishments, direct, analyst, improvement

Topic  1
s

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
management, busines, human, development, manager, resources, program, operations, manage, cost, performance, recruitment, market, quality, analysis, strategic, administration, develop, service, managed, resource, risk, executive, budget, proceses, policies, managers, compensation, implementation, relations, systems, suport, payrol, benefit, personel, strategy, client, employe, hire, division, organizational, delivery, data, improve, developed, compliance, senior, revenue, consultant, growth, test, insurance, functional, global, reduce, candidates, vendor, analyst, achieve, acros, technical, safety, maintain, enterprise, product, ontario, eficiency, initiatives, save, technology, value, operational, salary, activities, profesional, clients, proces, control, monitor, sucesfuly, drive, deliver, retention, services, talent, benefits, stafing, sucesful, marketing, improvement, created, multiple, information, labor, customer, pmo, acomplishments, los, stakeholders, direct

Topic  1


Topic  0
management, busines, human, development, manager, resources, program, operations, manage, cost, performance, recruitment, analysis, quality, strategic, administration, market, develop, service, risk, managed, resource, executive, policies, budget, proceses, managers, compensation, implementation, relations, payrol, benefit, systems, suport, personel, strategy, client, employe, hire, division, organizational, delivery, improve, data, developed, senior, compliance, revenue, consultant, growth, insurance, functional, global, candidates, reduce, test, analyst, vendor, achieve, acros, enterprise, ontario, product, eficiency, maintain, technical, initiatives, salary, save, operational, value, safety, technology, profesional, clients, proces, activities, drive, retention, sucesfuly, services, talent, deliver, benefits, monitor, stafing, sucesful, control, information, created, pmo, improvement, multiple, acomplishments, customer, los, labor, claim, stakeholders, direct

Topic  1
sal

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


> As we can see in the previous result some topics contains domain specific stop words so, we will remove some of them. 

### Fisrt Itration of Removing Domain Specific Word

In [55]:
stop_words = ['format', 'want', 'contact', 'adres', 'relevant', 'avoid', 'aplying', 
              'demonstrate', 'examples', 'employer', 'refer', 'reference', 'phone', 
              'start', 'colege', 'posible', 'sure', 'important', 'use', 'number', 'verbs', 
              'style', 'cover', 'include', 'action', 'clas', 'type', 'leter', 'kep', 
              'separate', 'font', 'draft', 'person', 'aply', 'acomplishments', 'thre', 'speak', 
              'way', 'read', 'ned', 'degre', 'employers', 'volunter', 'begin', 'past',
              'language', 'aditional', 'submit', 'guide', 'candidate',
              'clases', 'highlight', 'make', 'copy', 'oportunity',
              'think', 'aid', 'title', 'lok', 'location', 'organize', 'serve', 'print', 
              'supervise', 'atention', 'contribute', 'expect', 'seking', 'chronological',
              'ask', 'reverse', 'compile', 'white', 'explain', 'coursework',  'easy',
              'note', 'honors', 'brief', 'emphasize', 'erors', 'short', 'construct', 'writen', 
              'author', 'place', 'abroad', 'consider', 'gain', 'choose', 'advise', 
              'know', 'speling', 'internship', 'recent','asociation', 'research', 
              'curiculum', 'program', 'public', 'council', 
              'publications', 'comunity',  'awards',  
              'science', 'practice', 'asociate', 
              'presentations', 'academy', 'evaluation',  'social', 'chair',  'foundation', 
              'studies', 'activities',  'asesment', 'special', 
              'workshop', 'study', 'grant', 'human', 'work', 'world', 
              'felow', 'year', 'honors', 
              'undergraduate', 'life',  'chapter', 
              'seminar', 'non', 'private', 'policy', 'cordinator', 'curent',   
              'future', 'clas',  'conference', 
              'united', 'county', 'home', 'family', 'pres', 'symposium', 'forum', 'john',
              'isues', 'bok', 'case', 'series', 'scholarship', 'history', 
              'graduate', 'editor', 'outstanding', 'suport', 'learning', 'regional',
              'administrative',  'young', 'second', 'thre', 'court', 'initiative'
              'available', 'request', 'personal', 'english', 'references', 'template',
              'example', 'loking', 'info', 'question', 'read', 'qualifications', 'welcome', 
              'prior', 'websites',   'permision', 'download', 'distribute',
              'link', 'dayjob', 'copyright', 'able', 'ccopyright', 'areas', 'duties', 'key', 
              'expertise',  'competencies', 'hons', 'coventry',  'sumary', 
              'details',  'highly', 'nationality',  'jobsekers', 'profesional',
              'driving', 'birmingham', 'license', 'british', 'environment', 'central',
              'work', 'yes', 'suitable', 'nuneaton',  
              'senior', 'working', 'history',  
              'deliver',  'ambitious',  'having',
              'creative', 'providing', 'atention', 'people', 'coleagues', 'carer', 'drive',
              'extensive', 'particular', 'road', 'pose', 'contact', 'sekers', 'run',
              'dob', 'right', 'individual', 'prove', 
              'confident', 'diploma', 'asisting', 'motivate', 'big' ,'interpreted', 
              'arbitrated', 'shaped', 'invented', 'delegated', 'explained', 'surveyed', 
              'wrote', 'inspected',  'checked', 'calculated', 'compared', 
              'stimulated', 'repaired', 'weight', 'fabricated', 'scheduled', 'specialized', 
              'programed', 'contracted', 'adresed', 'educated', 'advertised', 'atained', 
              'modified', 'reinforced', 'examined', 'synthesized', 'acomplished', 'estimated',
              'criteria', 'solved', 'operated', 'directed', 'asembled',
              'influenced' , 'executed', 'persuaded', 'verified', 
              'adapted', 'computed', 'determined', 'fod', 'john', 'aranged', 'entertained', 'established', 
              'instituted', 'revised', 'separate', 'fel', 'furnished', 'originated', 'body',
              'status', 'conceived', 'aplicants', 'published', 'thank', 'unified', 'facilitated', 
              'guidelines', 'transmited', 'familiarized', 'say', 'wish', 'demonstrated', 
              'hired', 'discovered', 'indicate', 'instaled', 'anticipated', 'streamlined', 
              'projected', 'formulated', 'age', 'researched', 'underline', 'cover', 'asesed',
              'gathered', 'employer', 'proposed', 'sure', 'adreses', 'qualified', 'expanded',
              'detected', 'pionered', 'right', 'fashioned', 'clarified', 'indian', 'hindi', 'maried', 'male']

In [56]:
df_after_2 = remove_domain_stop_words(stop_words = stop_words, df_edit = df_after)

In [57]:
df_after_2

,TEXT,SPACY_TEXT
0,programing python pandas numpy scipy scikit learn matplotlib javascript jquery machine learn regresion svm bayes random forest decision tres bosting techniques cluster analysis word embeding sentiment analysis natural procesing dimensionality reduction topic modeling lda nmf pca neural nets database visualizations mysql sqlserver casandra hbase elasticsearch plotly kibana matplotlib gplot others regular expresion angular logstash kafka python flask git docker computer vision open dep learn data asurance asociatedata asurance ernst lpskil javascript jquery python monthscompany ernst fraud investigations dispute services asurance technology asisted review tar technology asists acelerating analytics generate develop automate platform tol scratch discovery domain tol predictive cod topic automate reduce labor cost spend lawyers understand flow solution development clasification predictive analysis mine information text data analyze output precision monitor entire tol tar asists predictive cod topic evidence folowing standards developed clasifier red flag fraud tols technologies python scikit learn tfidf word vec doc vec cosine similarity bayes lda nmf topic vader text blob sentiment analysis matplot lib dashboard multiple data analytic projects usa clients text analytics motor vehicle customer review data received customer fedback survey data sentiment positive negative neutral analysis customer coments acros categories created heat map term survey category frequency word extracted positive negative word acros survey categories plot word cloud created customize tableau dashboards visualizations chatbot developed friendly chatbot products simple operation reservation options chat bot entire product giving overview tol via platform give recomendation responses chain answer inteligence pipeline requirement recomended tols technologies python natural procesing nltk spacy topic sentiment analysis word embeding scikit learn javascript jquery sqlserver information governance organizations inform decisions information store integrate information governance portfolio synthesize inteligence acros unstructured data source facilitate organizations counter information risk scan data multiple source parse extract meta data information push index elastic search customize interactive dashboards kibana preforming rot analysis data give information data either redundant outdated trivial preforming ful text search analysis elastic search predefined methods tag personaly identifiable information security frequently target cyber atacks tols technologies python flask elastic search kibana fraud analytic platform fraud analytics investigative platform red flag fap fraud analytics investigative platform inbuilt manager suite analytics systems clients interogate acounting systems anomalies indicators fraud advance analytics tols technologies javascript sqlserver jquery botstrap node,"( , Programing, , Python, pandas, numpy, scipy, scikit, learn, matplotlib, Sql, Java, JavaScript, JQuery, Machine, learn, Regresion, SVM, Bayes, Random, Forest, Decision, Tres, Bosting, techniques, Cluster, Analysis, Word, Embeding, Sentiment, Analysis, Natural, Language, procesing, Dimensionality, reduction, Topic, Modeling, LDA, NMF, PCA, Neural, Nets, Database, Visualizations, Mysql, SqlServer, Casandra, Hbase, ElasticSearch, Plotly, kibana, matplotlib, gplot, Tableau, Others, Regular, Expresion, , Angular, Logstash, Kafka, Python, Flask, Git, Docker, computer, vision, Open, , Dep, learn, , Data, Science, Asurance, AsociateData, Science, Asurance, Asociate, Ernst, Young, LPSkil, , JAVASCRIPT, , jQuery, , Python, , monthsCompany, , Ernst, Young, , Fraud, Investigations, Dispute, Services, Asurance, ...)"
1,uit rgpvdata scientistdata scientist matelabsskil python les statsmodels aws les machine learn les sklearn les scipy les keras les monthscompany matelabs platform busines profesionals dumies enthusiasts block achievements tasks behind sukh sagar bengalu

In [58]:
tfidf2 = TfidfVectorizer(stop_words='english')
doc_words2 = tfidf2.fit_transform(df_after_2.TEXT)
tfidf_df2 = pd.DataFrame(doc_words2.toarray(),columns=tfidf2.get_feature_names())

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [59]:
topic_nums = list(np.arange(10, 20, 1))
for num in topic_nums:
    nmf = NMF(num)
    nmf.fit_transform(doc_words2)
    display_topics(nmf, tfidf2.get_feature_names(), 100)
    print("------------------------------------------")

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
management, busines, manager, development, operations, resources, manage, market, performance, cost, recruitment, service, quality, analysis, administration, strategic, budget, develop, managers, risk, executive, proceses, managed, client, implementation, resource, policies, clients, systems, data, relations, strategy, compensation, personel, information, payrol, delivery, benefit, revenue, acount, customer, maintain, marketing, improve, technical, growth, hire, employe, consultant, organizational, operational, developed, functional, compliance, insurance, analyst, division, global, financial, acros, value, technology, achieve, product, monitor, sucesful, vendor, reduce, test, enterprise, eficiency, control, sucesfuly, safety, relationships, available, initiatives, services, proces, relationship, candidates, save, multiple, stakeholders, salary, comunications, direct, ontario, retention, improvement, internal, consulting, complex, contract, campaign, objectives, oportunities,

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
management, busines, manager, development, resources, operations, manage, cost, market, performance, recruitment, service, administration, budget, strategic, quality, managers, executive, develop, policies, managed, relations, risk, resource, compensation, payrol, personel, proceses, analysis, clients, benefit, implementation, strategy, client, hire, marketing, acount, maintain, employe, delivery, information, systems, growth, customer, revenue, organizational, compliance, division, improve, operational, insurance, safety, developed, consultant, sucesful, value, monitor, candidates, available, achieve, control, eficiency, reduce, relationships, technical, sucesfuly, services, technology, global, salary, relationship, initiatives, financial, acros, save, vendor, campaign, retention, ontario, direct, legal, comunications, brand, talent, contract, multi, profile, product, stafing, benefits, objectives, employment, oportunities, productivity, labor, departments, improvement, sati

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


...


Topic  0
busines, management, development, manager, analysis, operations, risk, manage, client, strategic, functional, implementation, managed, proceses, technology, global, strategy, analyst, revenue, systems, executive, quality, enterprise, budget, clients, acount, delivery, growth, technical, cost, performance, consultant, consulting, software, product, operational, develop, pmo, administration, value, design, managers, stakeholders, proces, led, developed, complex, infrastructure, market, acros, integration, improve, resource, test, achieve, scope, cros, multiple, services, define, information, pmp, vendor, initiatives, dolar, international, data, division, relationships, multi, governance, relationship, documentation, partner, gather, save, impact, certified, sucesfuly, asia, leadership, sucesful, reduce, pacific, contract, solutions, oportunities, consult, portfolio, sap, establish, resources, aplication, vendors, comunications, created, mba, service, experienced, organizational

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
busines, management, development, manager, analysis, operations, risk, manage, client, strategic, implementation, managed, proceses, technology, functional, global, revenue, strategy, systems, analyst, data, executive, enterprise, budget, clients, acount, quality, growth, cost, technical, performance, delivery, consultant, consulting, operational, develop, pmo, administration, value, managers, developed, complex, infrastructure, market, led, product, design, stakeholders, proces, sap, improve, acros, software, achieve, multiple, resource, information, integration, scope, services, cros, initiatives, pmp, international, define, division, dolar, vendor, relationships, multi, governance, relationship, save, certified, reduce, sucesfuly, leadership, partner, oportunities, solutions, asia, pacific, consult, service, portfolio, establish, sucesful, contract, impact, resources, documentation, created, comunications, gather, insurance, experienced, vendors, mba, organizational, direc


Topic  0
management, busines, resources, development, manager, recruitment, operations, cost, performance, manage, administration, policies, payrol, compensation, resource, benefit, budget, managers, develop, personel, relations, quality, employe, strategic, executive, risk, hire, managed, proceses, candidates, implementation, compliance, systems, organizational, salary, division, insurance, safety, analysis, improve, delivery, benefits, reduce, ontario, operational, maintain, service, eficiency, legal, consultant, save, talent, developed, induction, stafing, vendor, claim, employment, value, client, labor, growth, information, strategy, los, monitor, achieve, contract, productivity, retention, initiatives, technical, services, selection, profile, sucesfuly, acros, xbc, improvement, atendance, cordinating, infrastructure, workforce, generalist, sucesful, exit, company, control, internal, orientation, clients, engagement, pmo, manufacture, enhance, objectives, revenue, manpower, abc, s

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Second Itration of Removing Domain Specific Word

In [60]:
stop_words_2 = ['resume', 'list', 'use', 'position',
                'job', 'present', 'include', 'special',
                'carer', 'adres', 'relevant', 'sumer', 'leter', 'fal',
                'state', 'parent','volunter', 'cover', 'wil',
                'section', 'comunity', 'member', 'objective','intern', 
                'society', 'title', 'make', 'clas', 'center', 'sample', 'august',
                'employer', 'internship', 'monash', 'page', 'graduate', 'profesional', 'asisted', 'language',
                'campus', 'honors', 'help', 'city', 'public', 'word', 'asociation', 'year', 'bachelor',
                'staf', 'contact','format', 'events', 'chicago', 'specific', 'date', 'social', 'nebraska', 
                'point', 'neds','project', 'use','team', 'end', 'create','proces','document','role',
                'experience', 'case', 'requirement', 'description', 'report', 'worked', 'used',
                'object', 'like', 'lead', 'source', 'custom', 'provide', 'enterprise', 'tol', 'hibernate',
                'new',  'team', 'representative', 'store', 'teritory', 
                'year','brand', 'experience', 'train', 'key', 'city', 'goals', 'profesional', 'years',
                'generate', 'present', 'regional', 'anual', 'quota', 'executive', 'representatives', 'lead',
                'achieved', 'build', 'skils', 'client', 'rep', 'oportunities', 'achievements', 
                'york', 'latin', 'region', 'achieve', 'texas', 'entries', 'knowledge', 
                'asociate', 'direct', 'industry', 'profitability', 'position', 'promotion', 'plan', 'consistently',
                'distribution', 'national', 'university', 'online', 'leadership', 'advertise',
                'kansas', 'profit', 'major', 'global', 'responsible', 'internet', 'bachelor', 'international',
                'statements','prepared', 'prepare', 'corporate', 'ledger', 'capital', 'company',
                'reconciliation', 'skils', 'experience','receivable','flow', 
                'internal', 'ability', 'preparation', 'entry', 'principles', 'parkland', 'general', 'project',
                'proces', 'plan', 'new', 'anual', 'quarterly','resume', 'cityland', 'review',
                'chicago', 'team', 'performed', 'work', 'cpa',
                'profesional', 'chartered', 'level', 'bachelor', 'fitzwiliam', 'email', 
                'strong', 'main', 'qualifications', 'performance', 'record', 'stret', 'senior', 'university'
                'exprience', 'details', 'months', 'les', 'description', 'maharashtra', 'year', 'company', 'pune',
                'january', 'monthscompany', 'mumbai', 'pvt', 'nagpur','india', 'june', 'universityhrskil', 
                'work', 'diploma', 'working', 'hibernate', 'worked', 'nashik', 'god', 'telangana', 'skils', 
                'requirement', 'learn', 'django', 'cricket', 'phule', 'savitribai', 'knowledge',
                'board', 'skil', 'amravati', 'secondary', 'university', 'role', 'servlet', 
                'institute', 'duration', 'title', 'solution', 'pradesh',
                'days', 'like', 'angular', 'job', 'solapur',
                'dayjob', 'personal', 'birmingham', 'template', 'coventry', 'able', 'use', 'info', 
                'available', 'work', 'skils', 'download', 'ability', 'distribute',
                'com', 'ccopyright', 'fod', 'copyright', 'make', 'permision', 'welcome', 
                'question', 'loking','prior', 'read', 'example', 'hons', 'cvs', 'page', 'help', 'nuneaton', 'gary',
                'company', 'duties','references', 'pas', 'jobsekers',
                'areas', 'relate', 'key', 'request', 'resume', 'competencies', 'ambitious', 'email',
                'driving', 'yes', 'restaurant', 'vyse', 'sekers', 'suitable', 'expertise', 'level',
                'qualifications', 'team', 'understand', 'pose', 'god', 'experience', 'maner',
                'knowledge', 'license', 'particular', 'highly', 'create', 'confident', 'nationality',
                'write', 'responsible', 'profesional', 'having', 'peg', 'staf', 'wel', 'excite', 
                'project', 'providing','project', 'record', 'environment',
                'train', 'company', 'performance',  'plan', 'implement', 'include', 'staf', 
                'team',  'new', 'resource', 'profesional', 'conduct', 'reduce', 'procedures',
                'relations', 'training', 'executive', 'personel', 'xae', 'senior', 'job', 'result', 
                'candidates', 'isues','department', 'division', 'improve','change', 'identify', 
                'leadership', 'lead', 'provide', 'time', 'work', 'level', 'save',
                'project', 'work','plant','team', 'specifications', 'plan', 'environmental','experience',
                'company', 'knowledge', 'include', 'india','phase', 'client',
                'complete', 'preparation', 'society','wind','scope', 'sub', 'completion', 
                'excelent', 'forte', 'god', 'skils', 'poses', 'key', 'air', 'structure', 'prepare',
                'farm','multitasking', 'agriculture', 'qualification', 'til', 'know', 'horticulture',
                'fluent', 'knowledge', 'profesional', 'template', 'florida', 'latin',
                'date', 'proficient', 'presentations', 'pest', 'gadgets', 'significant',
                'report', 'degre', 'achievement', 'player', 'resume', 'english', 'languages',
                'include', 'behavior', 'interests', 'skiled','various', 'cary', 'adept', 'bachelor', 'com',
                'crop', 'wild', 'extensive', 'stret', 'gmail', 'amigo',
                'nursery', 'supervisor', 'update', 'experience', 'miami', 
                'exceptionaly', 'wel', 'joye', 'diploma', 'harvest', 'won', 
                'mower', 'objective', 'sonata', 'carer', 'thompson', 'study', 'gren', 
                'display', 'understand','new', 'skils', 'qualifications', 'strong', 'open',
                'teler', 'cal', 'procedures', 'ability', 'work', 'knowledge',
                'check', 'phone', 'experience', 'asistant', 'excelent', 'client', 
                'god', 'georgia', 'representative', 'telers', 'daily', 'various', 
                'objective', 'asisted', 'sumary', 'profesional', 'provide', 'request', 'jersey', 
                'parkland', 'personalize', 'cals', 'policies', 'reference', 'cityland',
                'efective', 'position', 'handle', 'center', 'provided', 'carer',
                'handled', 'seking', 'fod', 'com', 'acount', 'email',
                'high', 'orient', 'interpersonal', 'properties', 'order', 'furnish', 'train', 'handling',
                'adept','york', 'present', 'nevada' ,'experience', 'campaign', 'frelance', 'online', 'work',
                'clients', 'xad','include', 'gogle','new', 'mobile', 'create',
                'facebok', 'galery', 'layout', 'university','mac', 'model', 'concept', 'layouts',
                'concepts', 'present', 'monash', 'team', 'york', 'lead', 'created', 
                'california', 'corporate', 'twiter', 'company', 'usability',
                'ideas', 'sample', 'client', 'skils', 'direction','care', 
                'asociation', 'center', 'comunity', 'xad', 'practice', 'present', 
                'university', 'member', 'department', 'terminology', 'eagan', 'staf', 'schol'
                'american', 'provide', 'education', 'public', 'colege', 'new'
                'carolina', 'york', 'conference', 'board', 'group', 
                'state', 'ability', 'experience', 'knowledge', 'kovner', 'perform', 'cordinator', 
                'profesional', 'condition', 'train', 'family', 'home', 'biling', 'registered', 
                'record', 'general', 'seidman', 'work','anual', 'ray']

In [61]:
df_after_3 = remove_domain_stop_words(stop_words = stop_words_2, df_edit = df_after_2)

In [62]:
tfidf3 = TfidfVectorizer(stop_words='english')
doc_words3 = tfidf3.fit_transform(df_after_3.TEXT)
tfidf_df3 = pd.DataFrame(doc_words3.toarray(),columns=tfidf3.get_feature_names())
tfidf_df3

,aacademic,aaditeshwar,aakruti,aan,aardvark,aaron,aashish,aba,ababa,abacus,...,zuluaga,zulux,zuorski,zur,zurich,zusamenarbeit,zwart,zwischen,zwitserlot,zxy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
topic_nums = list(np.arange(10, 20, 1))
for num in topic_nums:
    nmf = NMF(num)
    nmf.fit_transform(doc_words3)
    display_topics(nmf, tfidf3.get_feature_names(), 100)
    print("------------------------------------------")

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
management, busines, manager, development, operations, market, manage, resources, cost, service, quality, recruitment, analysis, budget, administration, strategic, develop, managers, risk, proceses, managed, customer, marketing, implementation, strategy, data, information, systems, maintain, delivery, payrol, revenue, compensation, growth, technical, organizational, hire, benefit, operational, functional, consultant, developed, employe, monitor, control, analyst, compliance, insurance, sucesful, technology, product, value, acros, test, relationships, safety, sucesfuly, eficiency, financial, vendor, relationship, initiatives, stakeholders, services, multiple, comunications, multi, complex, documentation, production, retention, departments, ontario, consulting, objectives, contract, salary, target, improvement, timely, pmo, profile, digital, establish, exist, productivity, stafing, standards, infrastructure, cordinated, satisfaction, resolve, talent, cordinating, managerial, pa

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
busines, management, manager, development, market, operations, resources, manage, cost, analysis, recruitment, budget, strategic, administration, develop, risk, quality, managers, proceses, managed, implementation, data, strategy, marketing, revenue, systems, analyst, compensation, growth, functional, payrol, information, delivery, benefit, financial, test, organizational, consultant, technical, developed, operational, employe, product, technology, maintain, acros, compliance, sucesful, value, service, monitor, hire, insurance, control, vendor, relationships, initiatives, sucesfuly, stakeholders, comunications, complex, consulting, relationship, objectives, pmo, multiple, salary, contract, improvement, documentation, services, eficiency, multi, digital, production, infrastructure, director, partner, talent, target, stafing, define, led, ontario, execute, integration, establish, exist, departments, finance, sap, benefits, execution, legal, decision, profile, mba, productivity,

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


...


Topic  0
busines, management, development, analysis, operations, manager, risk, functional, manage, analyst, proceses, technology, implementation, quality, managed, systems, strategic, software, revenue, delivery, technical, budget, strategy, test, service, consulting, consultant, operational, data, stakeholders, growth, cost, information, pmo, complex, managers, integration, develop, design, value, product, led, administration, acros, developed, cros, gather, multiple, documentation, define, infrastructure, relationship, multi, relationships, pmp, experienced, services, exist, impact, governance, vendors, vendor, visio, dolar, solutions, sap, initiatives, pacific, asia, sdlc, aplication, users, town, sucesful, certified, sucesfuly, problems, establish, aproach, tols, agile, partner, comunications, contract, gap, portfolio, external, requirements, mba, methodologies, louis, uat, transition, aceptance, aplications, stakeholder, large, consult, leader, transformation

Topic  1
sales, pr

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
management, busines, manager, resources, development, operations, recruitment, manage, cost, administration, payrol, managers, budget, compensation, develop, quality, risk, proceses, benefit, strategic, employe, managed, systems, implementation, analysis, hire, organizational, delivery, information, compliance, operational, maintain, salary, insurance, service, consultant, eficiency, ontario, growth, value, developed, legal, benefits, technical, strategy, vendor, talent, monitor, safety, stafing, induction, employment, productivity, technology, labor, control, acros, revenue, initiatives, sucesful, contract, services, pmo, claim, sucesfuly, multi, profile, retention, stakeholders, improvement, manpower, los, cordinating, selection, complex, relationship, workforce, atendance, objectives, ful, engagement, acounts, documentation, functional, managerial, manufacture, enhance, timely, comunications, establish, infrastructure, departments, exit, led, xbc, consulting, generalist, r


Topic  0
busines, management, development, analysis, analyst, risk, data, operations, functional, technology, proceses, manage, systems, implementation, manager, managed, strategic, revenue, consulting, technical, strategy, service, consultant, quality, delivery, information, design, sap, software, complex, operational, growth, stakeholders, budget, pmo, integration, value, led, managers, gather, developed, develop, cros, multiple, administration, experienced, acros, infrastructure, documentation, define, product, cost, users, relationships, impact, relationship, visio, exist, solutions, services, tols, multi, pmp, pacific, sdlc, town, governance, vendors, dolar, asia, louis, initiatives, certified, problems, comunications, mba, sucesfuly, vendor, partner, gap, sas, requirements, aproach, agile, consult, methodologies, establish, portfolio, aceptance, xbe, expert, external, market, defined, aplication, transformation, sucesful, microsoft, contract, large

Topic  1
sales, products, pri

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [64]:
nmf = NMF(13)
doc_topic_nmf = nmf.fit_transform(doc_words3)
display_topics(nmf, tfidf3.get_feature_names(), 100)

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(



Topic  0
management, busines, manager, development, resources, market, operations, manage, cost, recruitment, administration, budget, strategic, managers, develop, quality, risk, marketing, managed, payrol, strategy, compensation, analysis, proceses, implementation, growth, benefit, revenue, organizational, employe, delivery, hire, information, systems, operational, sucesful, consultant, service, compliance, developed, maintain, value, insurance, technology, monitor, technical, product, relationships, control, acros, sucesfuly, initiatives, vendor, salary, eficiency, multi, ontario, safety, relationship, comunications, digital, contract, objectives, talent, legal, stafing, benefits, services, profile, acounts, stakeholders, retention, productivity, functional, pmo, employment, establish, labor, departments, director, partner, induction, improvement, complex, multiple, grow, infrastructure, consulting, engagement, led, target, negotiation, timely, execute, ful, cordinating, claim, heal

/Users/shahadsulaiman/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [65]:
df_after_3.shape

(1255, 2)

### Data Labeling

In [66]:
topics_of_resume = pd.DataFrame(doc_topic_nmf, columns = ['Business_Administration', 'Sales_and_Marketing' , 
                                                          'Accounting' , 'Management_Information_System' , 
                                                          'Risk_Management' , 'Law' ,
                                                          'Electrical_and_Mechanical_Engineering', 
                                                          'Health' , 'Graphics_Design' , 'Finance' ,'Teaching' , 
                                                          'Development','hospitality'])
topics_of_resume['Topic_Name'] = topics_of_resume.idxmax(axis=1)
topics_of_resume

,Business_Administration,Sales_and_Marketing,Accounting,Management_Information_System,Risk_Management,Law,Electrical_and_Mechanical_Engineering,Health,Graphics_Design,Finance,Teaching,Development,hospitality,Topic_Name
0,0.004604,0.000000,0.006519,0.047213,0.000000,0.005506,0.002797,0.002996,0.000000,0.011755,0.010493,0.053640,0.022051,Development
1,0.004316,0.000000,0.000000,0.011818,0.000000,0.000000,0.006209,0.003686,0.007451,0.003604,0.002025,0.002219,0.000000,Management_Information_System
2,0.001034,0.006717,0.000000,0.009281,0.001545,0.000000,0.066354,0.000000,0.000000,0.000000,0.000000,0.060855,0.003184,Electrical_and_Mechanical_Engineering
3,0.000000,0.009743,0.008571,0.104175,0.000000,0.000000,0.000000,0.000000,0.002302,0.019421,0.001624,0.014840,0.017641,Management_Information_System
4,0.010037,0.000000,0.001204,0.015925,0.000000,0.000000,0.000000,0.000000,0.000000,0.003618,0.000000,0.015236,0.000000,Management_Information_System
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006570,0.000000,0.000000,0.096031,0.000000,0.113354,hospitality
1251,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166877,hospitality
1252,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002308,0.000000,0.000000,0.000000,0.000000,0.149363,hospitality
1253,0.014928,0.000000,0.000000,0.000000,0.000000,0.006750,0.002009,0.002006,0.000000,0.000000,0.038323,0.000000,0.051371,hospitality


### Store the Labeld Dataset into Pickle

In [67]:
write_pickle('Resumes.pickle', topics_of_resume)